In [1]:
import sys # for automation and parallelisation
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

In [2]:
import pandas as pd
import numpy as np
from quetzal.model import stepmodel
from quetzal.io import excel

In [3]:
model_path = '../model/' + scenario + '/'
input_path = '../input/'
input_static_path = '../input_static/'
output_path = '../output/'

In [4]:
# Load parameters for settings
params = excel.read_var(file='../input/parameters.xls', scenario=scenario)

# Trips per person from MiD2017

(Ortuzar 2011 [S. 166], Daly 1997): Define groups of individuals and determine for each group the number of trips per purpose.

Here the group of individuals is given by car availability and the urbanisation degree of the home zone. The deterimination of the number of trip per purpose is done by taking the mean value of number of reported trips per person in MiD2017. 

In [5]:
# Choose purposes for which volume data will be saved
purposes = ['shopping', 'errands', 'leisure', 'accompany']

In [6]:
# Load data on trips and persons from MiD survey
trips = pd.read_csv(input_path + 'transport_demand/calibration_all_trips_MiD2017.csv',
                    usecols=['P_ID', 'W_HOCH', 'urbanisation', 'purpose_model', 'car_avail',
                             'origin', 'destination', 'RegioStaR7'])
persons = pd.read_csv(input_path + 'transport_demand/MiD2017_Regional_Personen.csv', sep=';', decimal=',',
                      usecols=['HP_ID_Reg', 'P_HOCH', 'P_GEW', 'P_RBW_ANZ'])
# Define and rename relevant columns
persons = persons[['HP_ID_Reg', 'P_HOCH', 'P_GEW', 'P_RBW_ANZ']]
persons.columns = ['P_ID', 'P_HOCH', 'weight', 'num_rbw']
# clip person informations to trips table
trips = trips.merge(persons, on='P_ID')

In [7]:
# total trips per day (million)
# Only trips with specified origin and destination and other filters as in cal10
total_trips_MiD = trips['W_HOCH'].sum() / 1e6
total_trips_MiD

66.87158982564762

In [8]:
# Total population of Germany in 2017 (million)
total_pop_MiD = persons['P_HOCH'].sum() / 1e6
total_pop_MiD

82.17568400000002

In [9]:
# drop ureported car availability
trips = trips.loc[trips['car_avail'].isin([0,1])]
# set unreported number of regular business trips (regelmäßige berufliche Wege, rbw) to 0
rbw_dict = {n: 0 for n in [94, 99, 200, 607, 608]}
trips['num_rbw'] = trips['num_rbw'].replace(rbw_dict)
trips.shape

(254861, 11)

In [10]:
# get trips per person for each segment
urban_pop_dict = trips.drop_duplicates('P_ID').groupby('RegioStaR7').sum()['P_HOCH'].to_dict()
def trips_per_person(g):
    #share_inter = len(g.loc[g['origin']!=g['destination']]) / len(g)
    return g['W_HOCH'].sum() / urban_pop_dict[g['RegioStaR7'].unique()[0]]# * share_inter
segmented_trips_pp = trips.groupby(['RegioStaR7', 'purpose_model', 'car_avail']
                                  ).apply(trips_per_person)

## Save generation volumes in model zones

In [11]:
sm = stepmodel.read_json(model_path + 'de_zones')

In [12]:
# Load RegioStaR categories
regiostar = pd.read_excel(input_static_path + 'spatial_RegioStaR-PLZ.xlsx', sheet_name='ReferenzGebietsstand2022')
regiostar = regiostar.drop_duplicates('vbgemrs_22').set_index('vbgemrs_22')['RegioStaR7']
sm.zones['RegioStaR7'] = sm.zones['FID'].map(regiostar).fillna(77)

In [17]:
# Put trip generation in zones table
car_names = {0: '_no_car', 1: '_car'}
for p in purposes:
    for c in [0,1]:
        sm.zones[p+car_names[c]] = [segmented_trips_pp[(u,p,c)] * pop * 365
                                    for u, pop in zip(sm.zones['RegioStaR7'],
                                                      sm.zones['population'])]

In [20]:
sm.zones[[p+car_names[c] for p in purposes for c in [0,1]]].to_csv(
    model_path + 'generation_volumes.csv')

## Save inner-zonal probabilities by urbanisation degree

In [37]:
inner_shares = trips.groupby(['RegioStaR7', 'purpose_model', 'car_avail']).apply(
    lambda g: len(g.loc[g['origin']==g['destination']]) / len(g))
inner_shares = inner_shares.unstack('purpose_model').unstack('car_avail')
inner_shares.columns = [p+car_names[car] for p in inner_shares.columns.get_level_values(0)[::2] for car in [0,1]]
for u in inner_shares.index:
    inner_shares.loc[u] = inner_shares.loc[u].fillna(inner_shares.loc[u].mean())
    inner_shares.loc[u, inner_shares.loc[u]<=0.1] = inner_shares.loc[u].mean()
    inner_shares.loc[u, inner_shares.loc[u]>0.98] = inner_shares.loc[u].mean()

In [40]:
inner_shares.to_csv(input_path + 'inner-zonal_probabilities_agg_urban.csv')